In [10]:
!pip install minicons

In [11]:
import minicons
import pandas as pd
import numpy as np
import scipy
import random

In [12]:
data_synt_sb = list(pd.read_csv("./syn_data.csv")["synthetic"])
data_auth_ab = list(pd.read_csv("./aphasia_broca_processed.csv")["preprocessed_text"])
data_cont_ab = list(pd.read_csv("./control_broca_processed.csv")["preprocessed_text"])
data_cont_sb = list(pd.read_csv("./syn_data.csv")["preprocessed_text"])

data_synt_filt_sb = [s for s in data_synt_sb if len(s.split()) > 3]
data_auth_filt_ab = [s for s in data_auth_ab if len(s.split()) > 3]
data_cont_filt_ab = [s for s in data_cont_ab if len(s.split()) > 3]
data_cont_filt_sb = [s for s in data_cont_sb if len(s.split()) > 3]

random.seed(31345)

sample_size = min(len(data_synt_filt_sb), len(data_auth_filt_ab), len(data_cont_filt_ab))
data_auth_sample_ab = random.sample(data_auth_filt_ab, sample_size)
data_cont_sample_ab = random.sample(data_cont_filt_ab, sample_size)

indices = random.sample(range(len(data_synt_filt_sb)), sample_size)
data_synt_sample_sb = [data_synt_filt_sb[i] for i in indices]
data_cont_sample_sb = [data_cont_filt_sb[i] for i in indices]

print(len(data_auth_filt_ab))

4448


In [14]:
from minicons import scorer
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

import numpy as np

import json

model = scorer.IncrementalLMScorer('gpt2', 'cuda')

for data in [data_cont_sample_sb, data_cont_sample_ab, data_auth_sample_ab, data_synt_sample_sb]:
    scores = []
    stimuli_dl = DataLoader(data, batch_size = 50)
    for batch in tqdm(stimuli_dl):
        scores.extend([s.cpu().numpy() for s in model.sequence_score(batch, reduction = lambda x: -x.mean(0))])
    print(np.mean(scores))

100%|██████████| 89/89 [00:07<00:00, 11.29it/s]


4.756552


100%|██████████| 89/89 [00:13<00:00,  6.63it/s]


5.4323297


100%|██████████| 89/89 [00:10<00:00,  8.63it/s]


6.477992


100%|██████████| 89/89 [00:06<00:00, 14.46it/s]

6.389392


In [ ]:
model.token_score(data_synt_sample_sb[:30], surprisal = True, base_two = True)


In [15]:
model = scorer.MaskedLMScorer('roberta-base', 'cuda')

for data in [data_cont_sample_sb, data_cont_sample_ab, data_auth_sample_ab, data_synt_sample_sb]:
    scores = []
    stimuli_dl = DataLoader(data, batch_size = 10)
    for batch in tqdm(stimuli_dl):
        scores.extend([s.cpu().numpy() for s in model.sequence_score(batch, reduction = lambda x: -x.mean(0))])
    print(np.mean(scores))

100%|██████████| 445/445 [00:53<00:00,  8.27it/s]


2.9628186


100%|██████████| 445/445 [01:43<00:00,  4.32it/s]


3.5039625


100%|██████████| 445/445 [00:45<00:00,  9.71it/s]


5.0780425


100%|██████████| 445/445 [00:36<00:00, 12.32it/s]

5.5124855
